# Split dataset and upload to huggingface
* This notebook will spilt the dataset into: training, validate and test set.
* Then upload the dataset to huggingface 

In [2]:
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict
from huggingface_hub import notebook_login

In [3]:
name = "highest_vs_rest"
file_name = f"clean_{name}.csv"
df = pd.read_csv(file_name)
df

,text_clean,label
0,summary when trying to edit an existing conflu...,1
1,i could reproduce this errorbug in sourcetree ...,1
2,i get an exception when the application is fir...,0
3,system errora system error has occurredif this...,0
4,summary service desk email notifications are i...,0
...,...,...
9354,there is a erroneous notondemandcondition in t...,1
9355,note this bug report is for confluence server ...,1
9356,note this bug report is for jira server using ...,0
9357,app fails to start up the main windows is disp...,1


In [4]:
null_rows = df[df['text_clean'].isnull()]
null_rows

,text_clean,label


In [5]:
df = df.dropna(subset=['text_clean'])
# reset index
df = df.reset_index(drop=True)
df

,text_clean,label
0,summary when trying to edit an existing conflu...,1
1,i could reproduce this errorbug in sourcetree ...,1
2,i get an exception when the application is fir...,0
3,system errora system error has occurredif this...,0
4,summary service desk email notifications are i...,0
...,...,...
9354,there is a erroneous notondemandcondition in t...,1
9355,note this bug report is for confluence server ...,1
9356,note this bug report is for jira server using ...,0
9357,app fails to start up the main windows is disp...,1


In [6]:
null_rows = df[df['text_clean'].isnull()]
null_rows

,text_clean,label


In [7]:
df.label.value_counts().to_frame()[:50]
value_counts_df = df['label'].value_counts().to_frame()
value_counts_df

,count
label,
1,4680
0,4679


In [8]:
# Split dataframe into three parts: training, validation and testing.
# Currently 80/10/10 split.
def train_validate_test_split(df, train_percent=.8, validate_percent=.1, seed=42):
    np.random.seed(seed)
    # Shuffle index of dataframe
    perm = np.random.permutation(df.index)
    
    df_length = len(df.index)
    
    # Number of row in training set
    train_end = int(train_percent * df_length)
    # Number of rows in validate set
    validate_end = int(validate_percent * df_length) + train_end
    
    # From start to train end
    train = df.iloc[perm[:train_end]]
    # From train_end to validate_end
    validate = df.iloc[perm[train_end:validate_end]]
    # From validate to the last row in dataframe.
    test = df.iloc[perm[validate_end:]]
    return train, validate, test

In [9]:
# Split dataframe into three parts: training, validation and testing.
train , validate , test = train_validate_test_split(df)

In [10]:
# Convert from Pandas DataFrame to Hugging Face datasets
tds = Dataset.from_pandas(train)
vds = Dataset.from_pandas(validate)
test_ds = Dataset.from_pandas(test)

ds = DatasetDict()

ds["test"] = test_ds
ds["train"] = tds
ds["validate"] = vds

ds

DatasetDict({
    test: Dataset({
        features: ['text_clean', 'label', '__index_level_0__'],
        num_rows: 937
    })
    train: Dataset({
        features: ['text_clean', 'label', '__index_level_0__'],
        num_rows: 7487
    })
    validate: Dataset({
        features: ['text_clean', 'label', '__index_level_0__'],
        num_rows: 935
    })
})

In [ ]:
# Uncomment this if not logged in.
#notebook_login()

In [11]:
# Push to Hugging Face Hub
ds.push_to_hub(name)

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/8 [00:00<?, ?ba/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]